In [ ]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import matplotlib.pyplot as plt

# Code `get_data()`

## Create _draft code_ in the notebook

In [ ]:
# Where am I?
os.getcwd()

In [ ]:
import os
import pandas as pd

# Build relative path to csv folder 
csv_path = '/home/vincent/code/vtmoreau/04-Decision-Sicence/01-Project-Setup/data-context-and-setup/data/csv' # Bad practice!
csv_path = '../data-context-and-setup/data/csv' # Mac OS / Ubuntu only
csv_path = '..\\data-context-and-setup\\data\\csv' # Windows only
csv_path = os.path.join('..', 'data-context-and-setup','data','csv') # Robust to Linux/OS/Windows variations

csv_path

In [ ]:
file_names = [f for f in os.listdir(csv_path) if f.endswith('.csv')]
file_names

In [ ]:
#.strip() or .replace()
key_names = [key_name
    .replace('olist_','')
    .replace('_dataset','')
    .replace('.csv','')
    for key_name in file_names]
key_names

In [ ]:
# How to iterate over two list simulatenously ? 
# Let's discover how zip works :

l1 = ['a','b','c']
l2 = [1,2,3]
list(zip(l1,l2))

In [ ]:
# Create the dictionary

data = {}

for (k, f) in zip(key_names, file_names):
    data[k] = pd.read_csv(os.path.join(csv_path, f))

data['orders'].head()

## Transfer logic to `olist/data.py` 🐍

When you call a function that you will name `get_data` from `olist/data.py`, it should return the dictionary `data`:with key/value pairs `"name of the dataset":"dataset.csv"`

```python
from olist.data import Olist
Olist().get_data()
```
- Take your time to understand what happens when calling `Olist().get_data()`
- Your method `get_data()` needs to be callable from various places (e.g your Terminal, this notebook, another notebook located elsewhere, etc...)
- You can't use a relative path this time as the current working directory `os.getcwd()` depends on where you run the code in the first place


In [ ]:
# Test your code
from olist.data import Olist
Olist().get_data().keys()

# Let's build a business metric: `Weekly Sales` 📈

<img src="https://wagon-public-datasets.s3.amazonaws.com/04-Decision-Science/01-Project-Setup/olist_schema.png" width=1000>

**Goal**: Plot the weekly sum of `payment_value` for delivered orders

1️⃣ **Step 1**: Pseudo code your process in English below before actually coding

> YOUR ANSWER HERE

2️⃣ **Step 2**: Code your logic

In [ ]:
# YOUR CODE HERE

⌛️ Do not forget to handle `datetime` properly.

In [ ]:
# YOUR CODE HERE

👉 Set the `order_purchase_timestamp` as the index

In [ ]:
# YOUR CODE HERE

👉 You can now resample this dataset to compute metrics on a weekly basis.

In [ ]:
# YOUR CODE HERE

📈 Plot the weekly payment values to get a big picture of Olist's revenues.

In [ ]:
# YOUR CODE HERE